In [0]:
import zipfile
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
#download dataset
!wget http://detrac-db.rit.albany.edu/Data/DETRAC-train-data.zip

In [0]:
import zipfile
zip_ref = zipfile.ZipFile("DETRAC-train-data.zip", 'r')
zip_ref.extractall()
zip_ref.close()

In [0]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/yolo.zip", 'r')
zip_ref.extractall()
zip_ref.close()

ref = zipfile.ZipFile("/content/drive/My Drive/labels.zip", 'r')
ref.extractall()
ref.close()
!pip install terminaltables


In [0]:
from __future__ import division
import warnings
warnings.simplefilter("ignore", UserWarning)
from yolo import *
from yolo.models import *
from yolo.utils.logger import *
from yolo.utils.utils import *
from yolo.utils.datasets import *
from yolo.utils.parse_config import *
from yolo.test import evaluate
from google.colab import files
from terminaltables import AsciiTable
 
import os
import sys
import time
import datetime
import argparse

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

if __name__ == "__main__":

    epochs = 500
    batch_size =10
    gradient_accumulations = 2
    model_def="yolo/config/yolov3-custom.cfg"
    data_config = "yolo/config/custom.data"
    
    n_cpu=  2
    img_size = 416
    multiscale_training=  True
    compute_map=False
    
    epoch_drive=164000
    path = "/content/drive/My Drive/checkpoints/yolov3_ckpt_%d.pth"% epoch_drive
    checkpoints = torch.load(path)
    start_epoch = checkpoints["epoch"]+1
 
    print("start epoch...",start_epoch)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    data_config = parse_data_config(data_config)
    train_path = data_config["train"]
    valid_path = data_config["valid"]
    class_names = load_classes(data_config["names"])

    # Initiate model

  
    model = Darknet(model_def).to(device)
    model.apply(weights_init_normal)

    # If specified we start from checkpoint
    #[-----------------------------]
    print("loading pretrained model")
    model.load_state_dict(checkpoints["model_state_dict"])
    print("finishing")
        # Get dataloader
    dataset = ListDataset(train_path, augment=False, multiscale=multiscale_training)
    print("dataset len: ",len(dataset))
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=n_cpu,
        pin_memory=True,
        collate_fn=dataset.collate_fn,
    )

    optimizer = torch.optim.Adam(model.parameters())
    optimizer.load_state_dict(checkpoints['optimizer_state_dict'])
    
    metrics = [
        
        "loss",
        "x",
        "y",
        "w",
        "h",
        "conf",
        "cls"
      
    ]

    epochs = 200000
    path = "/content/drive/My Drive/yololog_%d.txt"% (start_epoch+5)

    dataloader_iter = iter(dataloader)
    message_s = ""
    print("start...")

    total_loss_print = 0
    total_loss_print_num = 0
    with open(path, "w") as log:

      for epoch in range(start_epoch,epochs):

          model.train()
          start_time = time.time()

          try:
              _, imgs, targets = next(dataloader_iter)
          except StopIteration:
              dataloader_iter = iter(dataloader)
              _, imgs, targets = next(dataloader_iter)

          imgs = Variable(imgs.to(device))
          targets = Variable(targets.to(device), requires_grad=False)

          loss, outputs = model(imgs, targets)
          loss.backward()

      
          optimizer.step()
          optimizer.zero_grad()


          log_str = "\n---- [Epoch %d] ----\n" % (epoch)

          metric_table = [["Metrics", *[f"YOLO Layer {i}" for i in range(len(model.yolo_layers))]]]

          # Log metrics at each YOLO layer
          message=""
          message = str(epoch)+" "
          for i, metric in enumerate(metrics):
              formats = {m: "%.6f" for m in metrics}
              formats["grid_size"] = "%2d"
              formats["cls_acc"] = "%.2f%%"
              row_metrics = [formats[metric] % yolo.metrics.get(metric, 0) for yolo in model.yolo_layers]
              #print("r_m",row_metrics)
              #print("m: ",metric," r_m:", *row_metrics)
              message += str(metric)+" "+str(float(row_metrics[0])+float(row_metrics[1])+float(row_metrics[2]))+" "
              metric_table += [[metric, *row_metrics]]

              # Tensorboard logging
              tensorboard_log = []
              for j, yolo in enumerate(model.yolo_layers):
                  for name, metric in yolo.metrics.items():
                      if name != "grid_size":
                          tensorboard_log += [(f"{name}_{j+1}", metric)]
              tensorboard_log += [("loss", loss.item())]
              #logger.list_of_scalars_summary(tensorboard_log, batches_done)
          total_loss_print += loss.item();
          total_loss_print_num += 1
          message +="\n"
          message_s += message
          log_str += AsciiTable(metric_table).table
          log_str += f"\nTotal loss {loss.item()}"

          log.write(message)
          log.flush()
          
          model.seen += imgs.size(0)
          if(epoch % 2000 == 0):
            print(message)
            print("tota;" ,total_loss_print/total_loss_print_num)
            total_loss_print_num = 0
            total_loss_print = 0
            
          if epoch % 4000 == 0:
             
              print("saving...")
              
              print(message)
              
              saving_file = {
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              
              }
              
              torch.save(saving_file, f"/content/drive/My Drive/checkpoints/yolov3_ckpt_%d.pth" % epoch)
              th_file = open("/content/drive/My Drive/yololog_message_%d.txt" % epoch, "w")
              th_file.write(message_s)
              th_file.close()
              message_s = ""
              

In [0]:
##--------------------EVAL-------------------------------
from __future__ import division
import warnings
warnings.simplefilter("ignore", UserWarning)
from yolo import *
from yolo.models import *
from yolo.utils.logger import *
from yolo.utils.utils import *
from yolo.utils.datasets import *
from yolo.utils.parse_config import *
from yolo.test import evaluate
from google.colab import files
from terminaltables import AsciiTable
 
import os
import sys
import time
import datetime
import argparse

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim

def evaluate(model, path, iou_thres, conf_thres, nms_thres, img_size, batch_size):
    model.eval()

    # Get dataloader
    dataset = ListDataset(path, img_size=img_size, augment=False, multiscale=False)
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=False, num_workers=1, collate_fn=dataset.collate_fn
    )

    Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

    labels = []
    sample_metrics = []  # List of tuples (TP, confs, pred)
    for batch_i, (_, imgs, targets) in enumerate(tqdm.tqdm(dataloader, desc="Detecting objects")):

        # Extract labels
        labels += targets[:, 1].tolist()
        # Rescale target
        targets[:, 2:] = xywh2xyxy(targets[:, 2:])
        targets[:, 2:] *= img_size

        imgs = Variable(imgs.type(Tensor), requires_grad=False)

        with torch.no_grad():
            outputs = model(imgs)
            outputs = non_max_suppression(outputs, conf_thres=conf_thres, nms_thres=nms_thres)

        sample_metrics += get_batch_statistics(outputs, targets, iou_threshold=iou_thres)

    # Concatenate sample statistics
    true_positives, pred_scores, pred_labels = [np.concatenate(x, 0) for x in list(zip(*sample_metrics))]
    precision, recall, AP, f1, ap_class = ap_per_class(true_positives, pred_scores, pred_labels, labels)
    print(precision, recall, AP, f1, ap_class)
    return precision, recall, AP, f1, ap_class





model_def="yolo/config/yolov3-custom.cfg"
data_config = "yolo/config/custom.data"

multiscale_training=  True
compute_map=False

epoch_drive=168000
path = "/content/drive/My Drive/checkpoints/yolov3_ckpt_%d.pth"% epoch_drive
checkpoints = torch.load(path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Get data configuration
data_config = parse_data_config(data_config)
train_path = data_config["train"]
valid_path = data_config["valid"]
class_names = load_classes(data_config["names"])

# Initiate model


model = Darknet(model_def).to(device)
model.apply(weights_init_normal)

model.load_state_dict(checkpoints["model_state_dict"])
print("finish load")


precision, recall, AP, f1, ap_class = evaluate(
                model,
                path=valid_path,
                iou_thres=0.5,
                conf_thres=0.5,
                nms_thres=0.5,
                img_size=416,
                batch_size=8,
            )
evaluation_metrics = [
    ("val_precision", precision.mean()),
    ("val_recall", recall.mean()),
    ("val_mAP", AP.mean()),
    ("val_f1", f1.mean()),
]

print(evaluation_metrics)

Detecting objects:   0%|          | 0/539 [00:00<?, ?it/s]

finish load


Computing AP: 100%|██████████| 1/1 [00:00<00:00, 14.35it/s]

[0.83225172] [0.90669523] [0.81970218] [0.86788003] [0]
[('val_precision', 0.8322517207472959), ('val_recall', 0.9066952329941081), ('val_mAP', 0.8197021765661301), ('val_f1', 0.8678800307613432)]


In [0]:
#detect image test

from __future__ import division
import time
import datetime
import argparse

from PIL import Image

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import warnings
warnings.simplefilter("ignore", UserWarning)
from yolo import *
from yolo.models import *
from yolo.utils.utils import *
from yolo.utils.datasets import *
from yolo.utils.parse_config import *
from yolo.test import evaluate
from terminaltables import AsciiTable
from google.colab.patches import cv2_imshow

from torchvision import transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.ticker import NullLocator
import cv2

import torchvision.transforms as transforms

path =  "/content/drive/My Drive/checkpoints/yolov3_ckpt_%d.pth"% 168000
checkpoints = torch.load(path)


data_config = "yolo/config/custom.data"
data_config = parse_data_config(data_config)
model_def = "yolo/config/yolov3-custom.cfg"
conf_thres = 0.6
nms_thres = 0.4
batch_size = 1
img_size=416


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs("output", exist_ok=True)
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
class_names = load_classes(data_config["names"])
print(1)

# Initiate model
model = Darknet(model_def).to(device)
model.apply(weights_init_normal)

model.load_state_dict(checkpoints["model_state_dict"])
model.eval()


im = cv2.imread('./Insight-MVT_Annotation_Train/MVI_20061/img00300.jpg')
im = cv2.resize(im,(416,416))
img = transforms.ToTensor()(im)
img = img.unsqueeze(0)
s_time = time.time()
detections = model(Variable(img.cuda()))
e_time=  time.time()

print("dff: ",e_time - s_time)
detections = non_max_suppression(detections, 0.5, 0.5)

for d in detections[0].cpu().detach().numpy():
  x1 = d[0]
  y1 = d[1]
  x2 = d[2]
  y2 = d[3]
  cv2.rectangle(im,(x1,y1),(x2,y2),(255,0,0),2)

cv2.imwrite("output/"+'out.jpg',im)




1
dff:  0.03576350212097168


True

In [0]:
!pip install filterpy

In [0]:
#track video for calculate accuracy

from sort import *

%matplotlib inline
from __future__ import division
import time
import datetime
import argparse

from PIL import Image

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import warnings
warnings.simplefilter("ignore", UserWarning)
from yolo import *
from yolo.models import *
from yolo.utils.utils import *
from yolo.utils.datasets import *
from yolo.utils.parse_config import *
from yolo.test import evaluate
from terminaltables import AsciiTable
from google.colab.patches import cv2_imshow

from torchvision import transforms
from PIL import Image
import cv2

warnings.simplefilter("ignore", UserWarning)
torch.set_default_tensor_type('torch.cuda.FloatTensor')
VIDEO_SAVE_DIR="traffic_result"
if(os.path.exists(VIDEO_SAVE_DIR)):
  !rm -r traffic_result
  os.mkdir(VIDEO_SAVE_DIR)
else:
  os.mkdir(VIDEO_SAVE_DIR)

COLORS = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
FONT = cv2.FONT_HERSHEY_SIMPLEX
start_epoch=50000
path =  "/content/drive/My Drive/checkpoints/yolov3_ckpt_%d.pth"% 168000
checkpoints = torch.load(path)

data_config = "yolo/config/custom.data"
data_config = parse_data_config(data_config)
model_def = "yolo/config/yolov3-custom.cfg"
conf_thres = 0.6
nms_thres = 0.4
batch_size = 1
img_size=416

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs("output", exist_ok=True)
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
class_names = load_classes(data_config["names"])

# Initiate model
model = Darknet(model_def).to(device)
model.apply(weights_init_normal)

model.load_state_dict(checkpoints["model_state_dict"])
model.eval()



frames = []
frame_count=0
mot_tracker = Sort()

the_file = open('test_tracking.txt')
list_of_test_img = []

for line in the_file:
  list_of_test_img.append(line.strip())


list_of_test_img = sorted(list_of_test_img)

all_ = []
path = "/content/Insight-MVT_Annotation_Train/"


for filename in list_of_test_img:
    frame = cv2.imread(path+str(filename),1)
    frame_count += 1
  
    im = cv2.resize(frame,(416,416))
    img = transforms.ToTensor()(im)
    img = img.unsqueeze(0)
    detections = model(Variable(img.cuda()))
    
    detections = non_max_suppression(detections, 0.5, 0.5)
    if(detections[0] != None):

      detections = detections[0].cpu().detach().numpy()
      if detections.size != 0:
      
        trackers = mot_tracker.update(detections)
      
        for d in trackers:
          try:
              xmin=int((d[0]/float(416)) * 960)
              ymin=int((d[1]/float(416))*540)
              xmax=int((d[2]/float(416))*960)
              ymax=int((d[3]/float(416))*540)
              label=int(d[4])
              cv2.rectangle(im,(xmin,ymin),(xmax,ymax),(255,0,0),2)
              cv2.putText(im, str(label), (int(xmin), int(ymin)),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
              #cv2_imshow(frame)
              #cv2.waitKey(1)
              all_.append([filename,label,xmin,ymin,xmax,ymax])
          except:
            print("error")            
    
    
    if(frame_count % 100==0):
      print(frame_count)
   

import pandas as pd

df = pd.DataFrame(all_, columns=["name","label","x1","y1",'x2','y2'])
df.to_csv('yolo_track_out.csv', index=False) 


In [0]:
!pip install filterpy

In [0]:
#track video

from sort import *

%matplotlib inline
from __future__ import division
import time
import datetime
import argparse

from PIL import Image

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import warnings
warnings.simplefilter("ignore", UserWarning)
from yolo import *
from yolo.models import *
from yolo.utils.utils import *
from yolo.utils.datasets import *
from yolo.utils.parse_config import *
from yolo.test import evaluate
from terminaltables import AsciiTable
from google.colab.patches import cv2_imshow

from torchvision import transforms
from PIL import Image
import cv2

warnings.simplefilter("ignore", UserWarning)
torch.set_default_tensor_type('torch.cuda.FloatTensor')
VIDEO_SAVE_DIR="traffic_result"
if(os.path.exists(VIDEO_SAVE_DIR)):
  !rm -r traffic_result
  os.mkdir(VIDEO_SAVE_DIR)
else:
  os.mkdir(VIDEO_SAVE_DIR)

COLORS = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]
FONT = cv2.FONT_HERSHEY_SIMPLEX
start_epoch=50000
path =  "/content/drive/My Drive/checkpoints/yolov3_ckpt_%d.pth"% 168000
checkpoints = torch.load(path)

data_config = "yolo/config/custom.data"
data_config = parse_data_config(data_config)
model_def = "yolo/config/yolov3-custom.cfg"
conf_thres = 0.6
nms_thres = 0.4
batch_size = 1
img_size=416

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs("output", exist_ok=True)
Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
class_names = load_classes(data_config["names"])

# Initiate model
model = Darknet(model_def).to(device)
model.apply(weights_init_normal)

model.load_state_dict(checkpoints["model_state_dict"])
model.eval()


$capture = cv2.VideoCapture("/content/drive/My Drive/checkpoints/b_traffic.mp4")

frames = []
frame_count=0
mot_tracker = Sort()

frame_path = []
for name in os.listdir("/content/Insight-MVT_Annotation_Train/MVI_40131"):
  frame_path.append("/content/Insight-MVT_Annotation_Train/MVI_40131/"+name)

frame_path = sorted(frame_path)
for path in frame_path:


# while True:
#     ret, frame = capture.read()
  
    # Bail out when the video file ends
    frame = cv2.imread(path)
    # if not ret:
    #     break        
    # Save each frame of the video to a list
    frame_count += 1

    im = cv2.resize(frame,(416,416))
    img = transforms.ToTensor()(im)
    img = img.unsqueeze(0)
    detections = model(Variable(img.cuda()))
    detections = non_max_suppression(detections, 0.5, 0.5)
    detections = detections[0].cpu().detach().numpy()
    if detections.size != 0:
     
      trackers = mot_tracker.update(detections)
    
      for d in trackers:
        try:
            xmin=int((d[0]/416) * 916)
            ymin=int((d[1]/416) * 540)
            xmax=int((d[2]/416) * 916)
            ymax=int((d[3]/416) * 540)
            label=int(d[4])
            cv2.rectangle(frame,(xmin,ymin),(xmax,ymax),(255,0,0),2)
            cv2.putText(frame, str(label), (int(xmin), int(ymin)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            #cv2_imshow(frame)
            #cv2.waitKey(1)
        except:
          print("error")            
      
    if(frame_count % 100==0):
      print(frame_count)
   

    name = '{}.jpg'.format(frame_count)
    name = os.path.join(VIDEO_SAVE_DIR, name)
    cv2.imwrite(name, frame)
    
    
    
    
    
    
##############CONVERT TRACKED IMAGE TO VIDEO###################

import glob
import os
import cv2
# video = cv2.VideoCapture("/content/drive/My Drive/checkpoints/b_traffic.mp4");


# # Find OpenCV version
# (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

# if int(major_ver)  < 3 :
#     fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
#     print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
# else :
#     fps = video.get(cv2.CAP_PROP_FPS)
#     print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
fps= 25
video.release();

def make_video(outvid, images=None, fps=30, size=None,
               is_color=True, format="FMP4"):
  
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        #print(image)
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid



images = list(glob.iglob(os.path.join("traffic_result", '*.*')))
# Sort the images by name index.
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))  

outvid =  "/content/drive/My Drive/yolo_out.mp4"
make_video(outvid, images, fps)